In [ ]:
cd ../data

In [4]:
import pandas as pd
import numpy as np

file_list = ['TPP.txt', 'TAM.txt','MLOE.txt', 'OKEWFSMP.txt', 'THWP.txt', 'TAMatter.txt', 'AIIMAT.txt']
outfile = open('corpus.txt', 'w')
for file in file_list:
  with open(file, 'r', encoding='ascii' ,errors='ignore') as filename:
    for line in filename:
      outfile.write(line)

## 1(c)ii

In [6]:
data = open('corpus.txt').read().lower()
data = set(data)
dic = {}
int_dic = {}
for i in data:
  dic[i] = float(ord(i)/126)
  int_dic[float(ord(i)/126)] = i
int_dic

{0.07936507936507936: '\n',
 0.25396825396825395: ' ',
 0.2619047619047619: '!',
 0.2698412698412698: '"',
 0.2777777777777778: '#',
 0.2857142857142857: '$',
 0.29365079365079366: '%',
 0.30158730158730157: '&',
 0.30952380952380953: "'",
 0.31746031746031744: '(',
 0.3253968253968254: ')',
 0.3333333333333333: '*',
 0.3412698412698413: '+',
 0.3492063492063492: ',',
 0.35714285714285715: '-',
 0.36507936507936506: '.',
 0.373015873015873: '/',
 0.38095238095238093: '0',
 0.3888888888888889: '1',
 0.3968253968253968: '2',
 0.40476190476190477: '3',
 0.4126984126984127: '4',
 0.42063492063492064: '5',
 0.42857142857142855: '6',
 0.4365079365079365: '7',
 0.4444444444444444: '8',
 0.4523809523809524: '9',
 0.4603174603174603: ':',
 0.46825396825396826: ';',
 0.47619047619047616: '<',
 0.48412698412698413: '=',
 0.49206349206349204: '>',
 0.5: '?',
 0.7222222222222222: '[',
 0.7301587301587301: '\\',
 0.7380952380952381: ']',
 0.746031746031746: '^',
 0.753968253968254: '_',
 0.769841269

In [7]:
tuple_list = []
for key, value in int_dic.items():
  tuple_list.append((key, value))

## 1(c) iii~v

In [8]:
W = 100
S = 1
input = []
output = []
data = open('corpus.txt').read().lower()
for i in range(0,len(data)-W+1):
  sequence1 = data[i:i+W-1]
  sequence2 = data[i+W-1]
  list1 = []
  for j in sequence1:
    list1.append(dic[j])
  input.append(list1)
  output.append(dic[sequence2])

from keras.utils import np_utils
new_input = np.reshape(input, (len(input), 99, 1))
new_output = np_utils.to_categorical(output)

## 1(c)vi~x

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint

LSTMmodel = Sequential()
LSTMmodel.add(LSTM(256, input_shape = (new_input.shape[1], new_input.shape[2])))
LSTMmodel.add(Dense(new_output.shape[1], activation='softmax'))
LSTMmodel.compile(loss='categorical_crossentropy', optimizer='adam')

filepath1 = '/content/drive/MyDrive/hw7/LSTM.weights.hdf5'
checkpoint = ModelCheckpoint(filepath = filepath1, monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
LSTMmodel.fit(new_input, new_output, epochs=30, batch_size=512, callbacks=[checkpoint])

Epoch 1/30
9901/9901 [==============================] - 277s 27ms/step - loss: 2.6173e-04
Epoch 2/30
9901/9901 [==============================] - 269s 27ms/step - loss: 2.5948e-05
Epoch 3/30
9901/9901 [==============================] - 269s 27ms/step - loss: 2.3909e-05
Epoch 4/30
9901/9901 [==============================] - 268s 27ms/step - loss: 2.5192e-05
Epoch 5/30
9901/9901 [==============================] - 269s 27ms/step - loss: 2.3791e-05
Epoch 6/30
9901/9901 [==============================] - 269s 27ms/step - loss: 2.4925e-05
Epoch 7/30
9901/9901 [==============================] - 270s 27ms/step - loss: 2.5127e-05
Epoch 8/30
9901/9901 [==============================] - 270s 27ms/step - loss: 2.5159e-05
Epoch 9/30
9901/9901 [==============================] - 270s 27ms/step - loss: 2.4547e-05
Epoch 10/30
9901/9901 [==============================] - 269s 27ms/step - loss: 2.3971e-05
Epoch 11/30
9901/9901 [==============================] - 269s 27ms/step - loss: 2.3963e-05
Epoch 12

In [11]:
print('best set of weights in terms of loss is' ,2.3791e-05)

best set of weights in terms of loss is 2.3791e-05


In [30]:
LSTMmodel.save("my_model")
reconstructed_model = keras.models.load_model("my_model")

text = 'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'
text = text.lower()
char_list = []
for i in text[0:99]:
  char_list.append(dic[i])

predict_list = []
for i in range(1000):
  new_text = np.reshape(char_list, (1, len(char_list), 1))
  predict = reconstructed_model.predict(new_text, verbose=0)
  index = np.argmax(predict)
  predict_list.append(tuple_list[index][1])

sentence = " "
for i in text[0:99]:
  sentence += i
for i in predict_list:
  sentence += i
print(sentence)

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


 there are those who take mental phenomena naively, just as they would physical phenomena. this schojjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj